In [3]:
!pip install -r requirements.txt

  ERROR: Could not find a version that satisfies the requirement torch===1.4.0 (from -r requirements.txt (line 6)) (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch===1.4.0 (from -r requirements.txt (line 6))


In [1]:
import matplotlib.pyplot as plt
import tifffile as tiff
import numpy as np
import os
import pandas as pd
import random

#Data information
sections = ['00', '01', '02', '03']
dates = ['20190606', '20190701', '20190706', '20190711', '20190721', '20190805', '20190815', '20190825', '20190909', '20190919', '20190924', '20191004', '20191103']
bands_all = 'B01 B02 B03 B04 B05 B06 B07 B08 B8A B09 B11 B12 CLD'.split(' ')
bands = 'B02 B03 B04 B08 B12 CLD'.split(' ')
kernel_size = 5

In [2]:
def open_image(path):
    return tiff.imread(path)

def sized_crop(image, pos, size=(32, 32)):
    return image[pos[0] - size[0] // 2:pos[0] + size[0] // 2 + 1, pos[1] - size[1] // 2:pos[1] + size[1] // 2 + 1]

In [3]:
#Get train df:
data = []
test = [] 
for section in sections:
    mask = open_image(f'data/{section}/{section[1]}_label.tif')
    fields = open_image(f'data/{section}/{section[1]}_field_id.tif')
    for date in dates:
        img_bands = {}
        for band in bands:
            img_bands[band] = open_image(f'data/{section}/{date}/{section[1]}_{band}_{date}.tif')
        for y in range(kernel_size, mask.shape[0] - kernel_size):
            for x in range(kernel_size, mask.shape[1] - kernel_size):
                    if fields[y, x] != 0:
                        data_slice = [section, date, mask[y, x], x, y]
                        for band in bands:
                            kernel_data = []
                            cropped_img = sized_crop(img_bands[band], (y, x), (kernel_size, kernel_size))
                            data_slice.append(cropped_img)
                        if mask[y, x] == 0:
                            test_slice = [fields[y, x]]
                            test_slice.extend(data_slice)
                            test.append(test_slice)
                        else:
                            test_slice = [fields[y, x]]
                            test_slice.extend(data_slice)
                            data.append(test_slice)

In [4]:
colnames = ['field_id', 'section', 'date', 'class', 'x', 'y']
colnames.extend(bands)
train_df = pd.DataFrame(data, columns=colnames)
test_df = pd.DataFrame(test, columns=colnames)

In [5]:
train_df.to_pickle('dataframes/train_data.csv')
test_df.to_pickle('dataframes/test_data.csv')